# Data cleaning

### Imports
Import libraries and write settings here.

In [87]:
%load_ext autoreload
%autoreload
%load_ext dotenv
%dotenv

import pandas as pd
import os
import qgrid
import ipywidgets
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


### Set up extractions from each nrcan csv

In [187]:
#extract target name from table name
def substring_table(tableName):
    start = tableName.find(": ") + len(": ")
    end = tableName.find(" Secondary")
    substring = tableName[start:end]
    return substring

#Concatenate table_id number
def write_table_id(index):
    table_id = 'tranca2017e_' + str(index)
    return table_id

#add additional columns to table
def add_target_metadata(df, targetName, table_id):
    df.insert(1, 'Target', targetName)
    df.insert(3, 'Units', 'Petajoules')
    df.insert(4, 'Table ID', table_id)
    return df

def add_source_metadata(df, sourceName, table_id):
    df.insert(0, 'Source', sourceName)
    df.insert(3, 'Units', 'Petajoules')
    df.insert(4, 'Table ID', table_id)
    return df

#extract only relevant data
#note locations are hardcodes for nrcan files types -- all are equivalent
def extract_relevant_data(df): 
    empty_index = df[df['Name'].isnull()].index.tolist()
    last_i = empty_index[next(x[0] for x in enumerate(empty_index) if x[1] > 10)]
    df = df.drop(df.iloc[:, 2:29], inplace = False, axis = 1)
    df = df.iloc[12:last_i,:]
    df = df.drop(0, axis = 1)
    df , hybrid = find_subtitle_row(df)
    #display(df)
    return df, hybrid

def find_subtitle_row(df):
    index = df[df['Name'].str.contains("(PJ)")].index
    if index != 12:
        hybrid = True
    else:
        hybrid = False
        df = df.drop(index, axis = 0)
    return df, hybrid

def configure_source(df, index, targetName):
    df.columns = ['Source', 'Value']
    table_id = write_table_id(index)
    df = add_target_metadata(df, targetName, table_id)
    return df

def configure_target(df, index, sourceName):
    df.columns = ['Target', 'Value']
    table_id = write_table_id(index)
    df = add_source_metadata(df, sourceName, table_id)
    return df

def configure_hybrid(df, index, otherNode, source):
    subT = df[df['Name'].str.contains('(PJ)')].index
    sub_i = int(subT[0])
    gen = df.iloc[0:(sub_i - 12),:]
    df = df.iloc[(sub_i - 11):,:]
    table_id = write_table_id(index)
    gen.columns = ['Target', 'Value']
    gen = add_source_metadata(gen, otherNode, table_id)
    if source == True:
        df.columns = ['Source', 'Value']
        df = add_target_metadata(df, otherNode, table_id)
    else:
        df.columns = ['Target', 'Value']
        df = add_source_metadata(df, otherNode, table_id)
    return df, gen

qgrid.enable()

### Loop Through Tables and append to master df

In [188]:
filePath = 'data/tranca2017e/tranCa2017e_'
net_df = pd.DataFrame(columns=['Source','Target','Value','Units','Table ID'])
net_gen = pd.DataFrame(columns=['Source','Target','Value','Units','Table ID'])

#index through all tables in idca2017e folder
for x in range(12,61):
    fileIndex = str(x)
    path = filePath + fileIndex + '.csv'
    df = pd.read_csv(path, encoding='cp1252', header=None)
    tableName = df.iloc[6,0]
    sourceName = substring_table(tableName)
    table_id = write_table_id(fileIndex)

    #Determine if this table is communicating data by Region, by Energy Source, or by Transportation Mode
    source = False
    if 'by Region' not in tableName:
        if 'by Energy Source' in tableName:
            source = True
            targetName = sourceName
    else:
        continue
    
    #extract first chunk of relevant data rows
    df.rename(columns={1:'Name'}, inplace=True)
    df, hybrid = extract_relevant_data(df)
    df = df.dropna(axis=1, how='all')
    if source == True and hybrid == False:
        df = configure_source(df, x, targetName)
    elif source == False and hybrid == False:
        df = configure_target(df, x, sourceName)
    else:
        df, gen = configure_hybrid(df, x, sourceName, source)
        net_gen = net_gen.append(gen)
    net_df = net_df.append(df)

/home/ssheeline/miniconda3/envs/supersankey/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
/home/ssheeline/miniconda3/envs/supersankey/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
/home/ssheeline/miniconda3/envs/supersankey/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
/home/ssheeline/miniconda3/envs/supersankey/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
/home/ssheeline/miniconda3/envs/supersankey/lib/python3.8/site-packages/pandas/core/strings.

In [189]:
df = net_df 
gen = net_gen

### Drop hidden / na values

In [196]:
# replace confidential labels and n/a with nan
#def clean_compiled(df):
df.replace("X", np.nan, inplace=True)
df.replace("n.a.", np.nan, inplace=True)

# Convert values to numeric
#df['Value'] = pd.to_numeric(df['Value'])
#return df

#df = clean_compiled(df)
#gen = clean_compiled(gen)

display(df)
display(gen)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

# Export data

In [197]:
#export detailed data
csvName = "tranca2017e_compiled.csv"
df.to_csv(csvName, index=False)

In [198]:
#export gen data
csvName2 = "tranca2017e_compiled_gen.csv"
gen.to_csv(csvName2, index=False)